## All Import Modules

In [1]:
import yfinance as yf
import datetime
import numpy as np
from ta import momentum, trend
import pandas as pd


In [2]:
import openpyxl
workbook = openpyxl.Workbook()
sheet = workbook.active

headers = ["Algo Name","SYMBOL","Entry Strick Price", "Entry Position", "Entry Date Time" , "SL","TARGET","Exit Position" ,"Exit Date Time","P&L","Exit Status","Tred Time"]
sheet.append(headers)


def Save_Data_in_Sheet(algo_name,symbol,entry_Strick_price,entry_position,entry_datetime,sl,target,exit_position,exit_datetime,pnl,exit_status):
    Tred_Time = (exit_datetime - entry_datetime ).total_seconds() / 60
    sheet.append([algo_name,symbol,entry_Strick_price,entry_position,entry_datetime,sl,target,exit_position,exit_datetime,pnl,exit_status,Tred_Time])
    workbook.save(f"ALGO LOGS.xlsx")
    # print("SAVE DATA")

## MENUPLIATE INPUT DATA

In [3]:
previes_days = 60
def Main_Dataframe(symbol,previes_days):
    # ------------------------------------ DAY TIME FRAME -----------------------------------------------------------
    ticker = yf.Ticker(symbol)
    day_df = ticker.history(period=f"80d", interval="1d")
    
    # Reset the index and convert the 'Date' column to date format
    day_df.reset_index(inplace=True)
    day_df['Date'] = day_df['Date'].dt.date
    
    # Calculate technical indicators
    day_df['Day_10EMA'] = trend.EMAIndicator(day_df['Close'], window=10).ema_indicator()
    
    day_df['Day_RSI'] = momentum.RSIIndicator(day_df['Close'], window=3).rsi()
    day_df['Day_RSI_EMA'] = trend.EMAIndicator(day_df['Day_RSI'], window=6).ema_indicator()

    # -----------ADD EXTRA INDICATORS HERE --------------
    day_df['Day_Trend'] = np.select(
    [day_df['Day_10EMA'] > day_df['Day_10EMA'].shift(1),
     day_df['Day_10EMA'] < day_df['Day_10EMA'].shift(1)],
    ['Uptrend', 'Downtrend'],'Sideways')
    # Rename columns and drop unnecessary columns
    day_df.rename(columns={'Open': 'Day_Open', 'High': 'Day_High', 'Low': 'Day_Low', 'Close': 'Day_Close'}, inplace=True)
    day_df.drop(columns=['Volume', 'Dividends', 'Stock Splits','Day_10EMA','Day_RSI'], inplace=True)
    day_df.dropna(inplace=True)

    # ------------------------------------ 5 MIN TIME FRAME -----------------------------------------------------------
    # previes_days = 5
    df = ticker.history(period=f"{previes_days}d", interval="5m")
    df.reset_index(inplace=True)
    df['Date'] = df['Datetime'].dt.date
    df['Time'] = df['Datetime'].dt.time
    
    df.drop(columns=['Volume', 'Dividends', 'Stock Splits','Datetime'], inplace=True)
    df.dropna(inplace=True)
    # -----------ADD EXTRA INDICATORS HERE --------------
    df['5EMA'] = trend.EMAIndicator(df['Close'], window=5).ema_indicator()
    df['15EMA'] = trend.EMAIndicator(df['Close'], window=15).ema_indicator()
    df['Candle'] = np.where(df['Close'] > df['Open'], 'Green', 'Red')
    
    df['Prev_5EMA'] = df['5EMA'].shift(1)
    df['Prev_15EMA'] = df['15EMA'].shift(1)

    df['RSI'] = momentum.RSIIndicator(df['Close'], window=6).rsi()
    df['RSI_EMA'] = trend.EMAIndicator(df['RSI'], window=12).ema_indicator()
    
   
    short_ema =trend.EMAIndicator(df["Close"], window=5)
    long_ema = trend.EMAIndicator(df["Close"], window=15)
    
    df["5EMA"] = short_ema.ema_indicator()
    df["15EMA"] = long_ema.ema_indicator()
    
    # Determine the perfect trend based on EMA crossovers
    df["Trend"] = "Sideways"  # Default to Sideways
    df.loc[(df["5EMA"] > df["15EMA"]) & (df["5EMA"].shift(1) <= df["15EMA"].shift(1)), "Trend"] = "Uptrend"
    df.loc[(df["5EMA"] < df["15EMA"]) & (df["5EMA"].shift(1) >= df["15EMA"].shift(1)), "Trend"] = "Downtrend"

    first_candle_high = df.groupby('Date')['High'].first()
    first_candle_low = df.groupby('Date')['Low'].first()
    
    # Create a new column for the first candle's high and low
    df['First_Candle_High'] = df['Date'].map(first_candle_high)
    df['First_Candle_Low'] = df['Date'].map(first_candle_low)

    # ------------------------------------ MERGE DATA FRAME -----------------------------------------------------------
    
    merged_df = day_df.merge(df, on='Date', how='inner')
    merged_df.dropna(inplace=True)
    merged_df['DateTime'] = pd.to_datetime(merged_df['Date'].astype(str) + ' ' + merged_df['Time'].astype(str))
    merged_df.drop(['Date', 'Time'], axis=1, inplace=True)
    return round(merged_df,2)
    # return day_df
    # return df

data = Main_Dataframe("^NSEI",previes_days)
# data = Main_Dataframe("HDFCBANK.BO")
# data.tail(5)

In [4]:
all_columsn = ['Day_Open', 'Day_High', 'Day_Low', 'Day_Close', 'Day_RSI_EMA',
        'Open', 'High', 'Low', 'Close', '5EMA', '15EMA',
       'Prev_5EMA', 'Prev_15EMA', 'RSI', 'RSI_EMA', 'First_Candle_High', 'First_Candle_Low']
day_all_columsn = ['Date', 'Day_Open', 'Day_High', 'Day_Low', 'Day_Close', 'Day_RSI_EMA']
# data.iloc[-2]

# Add Signals Columns

In [5]:
def Add_Signals(dfnew):
    df = dfnew.copy()
    df['Signal_Day_RSI'] = np.select([df['Day_RSI_EMA'] <= 30, df['Day_RSI_EMA'] >= 70],['CE', 'PE'],default='0')
    df['Signal_RSI_CE'] = np.where(df['RSI_EMA'] <= 30, 'CE', '0')
    df['Signal_RSI_PE'] = np.where(df['RSI_EMA'] >= 70, 'PE', '0')


    ce_5ema = (df['High'].shift(1) < df['5EMA'].shift(1)) & (df['Close'] >= df['5EMA'])
    pe_5ema = (df['Low'].shift(1) > df['5EMA'].shift(1)) & (df['Close'] <= df['5EMA'])
    
    df['Signal_5EMA'] = np.select([ce_5ema, pe_5ema],['CE', 'PE'], default='NONE')

    ce_ema_crossover = (df['5EMA'] > df['15EMA']) & (df['5EMA'].shift(1) <= df['15EMA'].shift(1))
    pe_ema_crossover = (df['5EMA'] < df['15EMA']) & (df['5EMA'].shift(1) >= df['15EMA'].shift(1))
    
    df['Signal_EMA_CROSSOVER'] = np.select([ce_ema_crossover, pe_ema_crossover],['CE', 'PE'], default='0')
    
    df['SL_Price'] = list(zip(df['High'].shift(1), df['Low'].shift(1) ))
    
    # df.drop(columns = all_columsn,inplace= True)
    return df
newdata = Add_Signals(data)
newdata.iloc[-1]

Day_Open                            20194.1
Day_High                           20291.55
Day_Low                             20183.7
Day_Close                           20267.9
Day_RSI_EMA                           90.89
Day_Trend                           Uptrend
Open                                20263.3
High                               20270.35
Low                                 20249.0
Close                              20255.75
5EMA                               20262.13
15EMA                              20265.54
Candle                                  Red
Prev_5EMA                          20265.32
Prev_15EMA                         20266.93
RSI                                   32.65
RSI_EMA                               44.36
Trend                              Sideways
First_Candle_High                  20225.75
First_Candle_Low                    20188.3
DateTime                2023-12-01 15:25:00
Signal_Day_RSI                           PE
Signal_RSI_CE                   

# ENTRY LOGIC

In [6]:
def CE_ENTRY_LOGIC(row):
    status = row['Signal_5EMA'] == "CE"
    return status
    
def PE_ENTRY_LOGIC(row):
    status = row['Signal_5EMA'] == "PE"
    return status

# STOPLOSS AND TARGET AND TRAILING

In [7]:
def GET_STOPLOSS_PRICE(row, status):
    n = 1 if status == "CE" else 0
    return row["SL_Price"][n]

def GET_TARGET_PRICE(row, status):
    
    n = 1 if status == "CE" else 0
    return 5 * (abs(row["SL_Price"][n] - row['Close'])) + row['Close']


# LOGIC

In [8]:
PE_POSITION = False
PE_ENTRY_PRICE = 0
PE_ENTRY_DATETIME = None
PE_STOPLOSS = 0
PE_TARGET = 0
def Entry_PE(row):
    global PE_POSITION, PE_ENTRY_PRICE, PE_STOPLOSS, PE_TARGET, PE_ENTRY_DATETIME
    Current_Price = row['Close']
            
    # ENTRY
    if PE_ENTRY_LOGIC(row):
        strick_price = int(round(row['Close'], -2))
        if not PE_POSITION: 
            # Declaration
            PE_POSITION = True
            PE_ENTRY_PRICE = Current_Price
            PE_ENTRY_DATETIME = row['DateTime']
            PE_STOPLOSS = GET_STOPLOSS_PRICE(row ,'PE')
            PE_TARGET = GET_TARGET_PRICE(row ,'PE')
            
    # EXIT SL
    if PE_POSITION and Current_Price >= PE_STOPLOSS:  
        Current_PNL = PE_ENTRY_PRICE - Current_Price        
        
        algo_name = "5 EMA"
        symbol = "PE"
        entry_Strick_price = round(PE_ENTRY_PRICE, -2)
        entry_position = PE_ENTRY_PRICE
        entry_datetime = PE_ENTRY_DATETIME
        sl = PE_STOPLOSS
        target = PE_TARGET
        exit_position = Current_Price
        exit_datetime = row['DateTime']
        pnl = Current_PNL
        exit_status = "SL"
        Save_Data_in_Sheet(algo_name, symbol, entry_Strick_price, entry_position, entry_datetime, sl, target, exit_position, exit_datetime, pnl, exit_status)

        # Declaration
        PE_POSITION = False
        PE_STOPLOSS = 0
        PE_TARGET = 0
        PE_ENTRY_PRICE = 0
            
    # EXIT TARGET
    if PE_POSITION and Current_Price <= PE_TARGET:
        Current_PNL = PE_ENTRY_PRICE - Current_Price 
        
        
        
        algo_name = "5 EMA"
        symbol = "PE"
        entry_Strick_price = round(PE_ENTRY_PRICE, -2)
        entry_position = PE_ENTRY_PRICE
        entry_datetime = PE_ENTRY_DATETIME
        sl = PE_STOPLOSS
        target = PE_TARGET
        exit_position = Current_Price
        exit_datetime = row['DateTime']
        pnl = Current_PNL
        exit_status = "TARGET"
        Save_Data_in_Sheet(algo_name, symbol, entry_Strick_price, entry_position, entry_datetime, sl, target, exit_position, exit_datetime, pnl, exit_status)

        # Declaration
        PE_POSITION = False
        PE_STOPLOSS = 0
        PE_TARGET = 0
        PE_ENTRY_PRICE = 0


    


In [9]:
CE_POSITION = False
CE_ENTRY_PRICE = 0
CE_ENTRY_DATETIME = None
CE_STOPLOSS = 0
CE_TARGET = 0
def Entry_CE(row):
    global CE_POSITION, CE_ENTRY_PRICE, CE_STOPLOSS, CE_TARGET, CE_ENTRY_DATETIME
    Current_Price = row['Close']
    
    if CE_ENTRY_LOGIC(row):
        strick_price = int(round(row['Close'], -2))
        
        # ENTRY
        if not CE_POSITION:  
            # Declaration
            CE_POSITION = True
            CE_ENTRY_PRICE = Current_Price
            CE_ENTRY_DATETIME = row['DateTime']
            CE_STOPLOSS = GET_STOPLOSS_PRICE(row,"CE")                
            CE_TARGET = GET_TARGET_PRICE(row ,'CE')
            
    # EXIT SL
    if CE_POSITION and Current_Price <= CE_STOPLOSS:  
        Current_PNL = Current_Price - CE_ENTRY_PRICE
        
        
        algo_name = "5 EMA"
        symbol = "CE"
        entry_Strick_price = round(CE_ENTRY_PRICE, -2)
        entry_position = CE_ENTRY_PRICE
        entry_datetime = CE_ENTRY_DATETIME
        sl = CE_STOPLOSS
        target = CE_TARGET
        exit_position = Current_Price
        exit_datetime = row['DateTime']
        pnl = Current_PNL
        exit_status = "SL"
        Save_Data_in_Sheet(algo_name, symbol, entry_Strick_price, entry_position, entry_datetime, sl, target, exit_position, exit_datetime, pnl, exit_status)

        # Declaration
        CE_POSITION = False
        CE_STOPLOSS = 0
        CE_TARGET = 0
        CE_ENTRY_PRICE = 0

            
    # EXIT TARGET
    if CE_POSITION and Current_Price >= CE_TARGET:
        Current_PNL = Current_Price - CE_ENTRY_PRICE 
        
        algo_name = "5 EMA"
        symbol = "CE"
        entry_Strick_price = round(CE_ENTRY_PRICE, -2)
        entry_position = CE_ENTRY_PRICE
        entry_datetime = CE_ENTRY_DATETIME
        sl = CE_STOPLOSS
        target = CE_TARGET
        exit_position = Current_Price
        exit_datetime = row['DateTime']
        pnl = Current_PNL
        exit_status = "TARGET"
        Save_Data_in_Sheet(algo_name, symbol, entry_Strick_price, entry_position, entry_datetime, sl, target, exit_position, exit_datetime, pnl, exit_status)

        # Declaration
        CE_POSITION = False
        CE_STOPLOSS = 0
        CE_TARGET = 0
        CE_ENTRY_PRICE = 0

        
        



In [10]:
newdata.apply(Entry_CE, axis=1)
newdata.apply(Entry_PE, axis=1)

16      None
17      None
18      None
19      None
20      None
        ... 
4495    None
4496    None
4497    None
4498    None
4499    None
Length: 4484, dtype: object

In [11]:
df = pd.read_excel("ALGO LOGS.xlsx")

In [12]:
# df[df['Exit Status'] == "TARGET"]

In [13]:
# df[df['Exit Status'] == "SL"]

In [14]:
total_tred = df.shape[0]
total_tred

153

In [15]:
total_SL_tred = df[df['Exit Status'] == "SL"].shape[0]
total_TARGET_tred = df[df['Exit Status'] == "TARGET"].shape[0]

In [16]:
print("SL COUNT :",total_SL_tred)
print("TARGET COUNT :",total_TARGET_tred)
print("TOTAL PNL:",round(df['P&L'].sum(),2))


SL COUNT : 37
TARGET COUNT : 116
TOTAL PNL: 845.75


In [17]:
# SL COUNT : 111
# TARGET COUNT : 47
# TOTAL PNL: -343.2

In [18]:
# SL COUNT : 42
# TARGET COUNT : 131
# TOTAL PNL: 799.95